In [5]:
# Codify the chromosome

In [ ]:
# Download pdb files for structure to calculate the STRIKE score

In [2]:
# Create population

In [3]:
# Crossover function

In [4]:
# Mutation function

In [6]:
# Three Objective functions: STRIKE, Totally conserved columns and Percentage of Non-gaps

In [ ]:
# MOSAStrE function - This is the main function that implements proposed algorithm